In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
import pickle
import batman

In [ ]:
#0params.t0
#1params.per
#2params.rp
#3params.a
#4params.inc

In [ ]:
#TESS: Two limb darkening coefficients
#5u1
#6u2

#ASIAGO: Two limb darkening coefficients
#7u1 TASTE
#8u2 TASTE

#TASTE polynomail trend
#9zero-th for the polynomial
#10first order
#11second order

#12jitter for TESS
#13jitter for TASTE

In [ ]:
theta = np.empty(14)

theta[0] = #time of inferior conjunction
theta[1] = #orbital period
theta[2] = #planet radius
theta[3] =  #semi-major axis
theta[4] =  #orbital inclination
theta[5] = 0.35 #TESS LD coeff u1
theta[6] =  #TESS LD coeff u2
theta[7] =  #TASTE LD coeff u1
theta[8] = #TASTE LD coeff u2
theta[9] = np.average(...TASTE lightcurve)  #zero-th order coefficient for the polynomial trend
theta[10] = 0.0 #first order coefficient for the polynomial trend
theta[11] = 0.0 #second order coedfficient for the polynomial trend
theta[12] = 0.0 #jitter parameter for TESS data
theta[13] = 0.0 #jitter parameter for TASTE data

In [ ]:
taste_bjd_tdb = pickle.load(open('taste_bjdtdb.p', 'rb'))


In [ ]:
##BY NEXT WEDNESDAY, PLEASE HAVE A PROPER TASTE LIGHT CURVE
##AT LEAST ONE TESS SECTOR ALREADY FILTERED

##AND ALL PARAMETERS SAVED IN THETA ARRAY.

Wednesday, 17 Dec

In [ ]:
import batman
params = batman.TransitParams()
params.t0 = theta[0]
params.period = theta[1]
params.rp = theta[2]
params.a = theta[3]
params.inc = theta[4]
params.ecc = 0 #decided to fix it to 0. does not need to be fixed by MCMC
params.w = 90
params.u = [theta[5], theta[6]] #for TESS
params.limb_dark = "quadratic"

m_tess = batman.TransitModel(params, tess_bjd_tdb)
tess_flux = m_tess.lightcurve

params.u = [theta[7],theta[8]]
m_taste = batman.TransitModel(params, taste_bjd_tdb)

median_bjdtdb_taste = np.median(taste_bjd_tdb)
polynomial_trend = theta[9] + theta[10]*(taste_bjd_tdb-median_bjd) 
                + theta[11]*(taste_bjd_tdb-median_bjd)**2


m_taste = m_taste.light_curve(params) * polynomial trend

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(taste_bjd_tdb, differential_allref, s=2)
plt.plot(taste_bjd_tdb, taste_flux, lw=2, c='C1')
plt.xlabel("BJD TDB")
plt.ylabel("Relative flux")
plt.show()

plt.figure(figsize=(6,4))
plt.scatter(tess_bjd_tdb, tess_normalized_flux, s=2)
plt.plot(tess_bjd_tdb, tess_flux, lw=2, c='C1')
plt.xlabel("BJD TDB")
plt.ylabel("Relative flux")
plt.show()

In [ ]:
tess_errors_with_jitter = tess_normalized_ferr**2 + theta[12]**2
taste_errors_with_jitter = differential_allref_error**2 + theta[13]**2

N = len(tess_errors_with_jitter) + len(taste_errors_with_jitter)

chi2_tess = np.sum( (tess_normalized_flux-tess_flux)**2 / tess_errors_with_jitter)
chi2_taste = np.sum( (differential_allref-taste_flux)**2 / taste_errors_with_jitter)
sum_ln_sigma_tess = np.sum(np.log(tess_errors_with_jitter))
sum_ln_sigma_taste = np.sum(np.log(taste_errors_with_jitter))

log_likelihood = -0.5 * ( N * np.log(2*np.pi) + chi2_tess + chi2_taste + sum_ln_sigma_tess + sum_ln_sigma_taste)
print('log_likelihood', log_likelihood)

In [ ]:
##his log_likelihood was ~83134

In [ ]:
def log_likelihood(theta):

    params = batman.TransitParams()
    params.t0 =  theta[0]                
    params.per = theta[1]                    
    params.rp =  theta[2]                 
    params.a =   theta[3]                   
    params.inc =  theta[4]    
    params.ecc = 0.
    params.w = 90. 
    params.u = [theta[5] , theta[6]]
    params.limb_dark = "quadratic"

    m_tess = batman.TransitModel(params, tess_bjd_tdb)    #initializes model
    tess_flux =m_tess.light_curve(params)          #calculates light curv

    params.u = [theta[7] , theta[8]]
    median_bjd = np.median(taste_bjd_tdb)
    polynomial_trend = theta[9]+theta[10]*(taste_bjd_tdb-median_bjd) + theta[11]*(taste_bjd_tdb-median_bjd)**2

    m_taste = batman.TransitModel(params, taste_bjd_tdb)    #initializes model
    taste_flux = m_taste.light_curve(params) * polynomial_trend

    
    tess_errors_with_jitter = tess_normalized_ferr**2 + theta[12]**2
    
    taste_errors_with_jitter = differential_allref_error**2 + theta[13]**2

    N = len(tess_errors_with_jitter) + len(taste_errors_with_jitter)

    chi2_tess = np.sum( (tess_normalized_flux-tess_flux)**2 / tess_errors_with_jitter)
    chi2_taste = np.sum( (differential_allref-taste_flux)**2 / taste_errors_with_jitter)

    sum_ln_sigma_tess = np.sum(np.log(tess_errors_with_jitter))
    sum_ln_sigma_taste = np.sum(np.log(taste_errors_with_jitter))

    log_likelihood = -0.5 * ( N * np.log(2*np.pi) + chi2_tess + chi2_taste + sum_ln_sigma_tess + sum_ln_sigma_taste)
    return log_likelihood

In [ ]:
# prior on c1: 0.59 \pm 0.10
from scipy import stats

x_range=np.arange(0.00, 1.00, 0.001)
y1_plot = stats.norm.pdf(x_range, loc=0.58, scale=0.10)
y2_plot = stats.norm.pdf(x_range, loc=0.35, scale=0.10)
plt.figure(figsize=(5,5))
plt.plot(x_range, y1_plot)
plt.plot(x_range, y2_plot)
plt.xlabel('Parameter value')
plt.ylabel('Probability Density Function')
plt.show()


In [ ]:
def log_prior(theta):
    prior = 0
    prior += np.log(stats.norm.pdf(theta[5], loc = 0.35, scale = 0.10))
    prior += np.log(stats.norm.pdf(theta[6], loc = 0.35, scale = 0.10))
    prior += np.log(stats.norm.pdf(theta[7], loc = 0.35, scale = 0.10))
    prior += np.log(stats.norm.pdf(theta[8], loc = 0.35, scale = 0.10))
    return prior

In [ ]:
print(log_prior(theta)) #he got 5.534586

In [ ]:
theta[0] = 2459500.53574  #time of inferior conjunction
theta[1] = 3.3366510632   #orbital period
theta[2] = 0.0764         #planet radius (in units of stellar radii)
theta[3] = 13.94          #semi-major axis (in units of stellar radii)
theta[4] = 88.9           #orbital inclination (in degrees)
theta[5] = 0.35           # TESS LD coeff u1
theta[6] = 0.23           # TESS LD coeff u2
theta[7] = 0.58           # TASTE LD coeff u1
theta[8] = 0.12           # TASTE LD coeff u2
theta[9] = 0.245           # zero-th order coefficient for the polynomial trend
theta[10] = 0.0           # first order coefficient for the polynomial trend
theta[11] = 0.0           # second order coefficient for the polynomial trend
theta[12] = 0.0           # jitter parameter for TESS data
theta[13] = 0.0           # jitter parameter for TASTE data


boundaries = np.empty([2, len(theta)])

boundaries[:,0] = [theta[0]-0.5, theta[0]+0.5]
boundaries[:,1] = [theta[1]-0.5, theta[1]+0.5]
boundaries[:,2] = [0.0, 0.5]
boundaries[:,3] = [0.0, 20.]
boundaries[:,4] = [0.00, 90.0]
boundaries[:,5] = [0.00, 2.0]
boundaries[:,6] = [-1.0, 1.0]
boundaries[:,7] = [0.00, 2.0]
boundaries[:,8] = [-1.0, 1.0]
boundaries[:,9] = [0.00, 2.0]
boundaries[:,10] = [-1.0, 1.0]
boundaries[:,11] = [-1.0, 1.0]
boundaries[:,12] = [0.0, 0.05]
boundaries[:,13] = [0.0, 0.05]

In [ ]:
def log_probability(theta):
    sel = (theta<boundaries[0,:]) | (theta > boundaries[1,:])
    if np.sum(sel) > 0:
        return -np.inf

    log_prob = log_prior(theta)
    log_prob += log_likelihood(theta)
    return log_prob

In [ ]:
##his log_prob was 86169.447

In [ ]:
nwalkers = 50
nsteps = 1000
ndim = len(theta)
import emcee

In [ ]:
starting_point = theta + np.abs(10**(-5) * np.random.rand(nwalkers, ndim))

In [ ]:
np.shape(starting_point) #should be 50,14

In [ ]:
from multiprocessing import Pool
with Pool() as pool:
    sampler = emcee.EnsembleSampler(nwalkers,
                                    ndim, 
                                    log_probability, 
                                    pool=pool 
                                   )
    sampler.run_mcmc(starting_point, nsteps, progress=True)
    

In [ ]:
pickle.dump(sampler, open('emcee_sampler_first_run.p', 'wb'))